### This notebook describes how to structure the project.

During the competition there were many tasks to do, plan, research, test, tune, accept disappointment of failed tests, and be happy with a little improvement. 

Therefore, the workspace and the files of the project should be structured in a flexible way with less repeated code. In other word, is to split code from data/configuration to save time and reduce errors/bugs and avoid copy/paste.

Therefore, first we define the entities in the project.
There are four main entities and I think these will be the same in all projects: experiment, model, level, and stack. These will be modeled by classes as described in this notebook.


*This is a draft work, and will be improved regularly.*


I would like to give credits to many kernels and websites among them:

 - Good introduction introduction about stacking: https://mlwave.com/kaggle-ensembling-guide/
 - https://www.kaggle.com/getting-started/18153#post103381
 - 


In [8]:
# Familiar imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display

import os
import glob
from datetime import datetime
from pathlib import Path




# For ordinal encoding categorical variables, splitting data, pipeline, and so on
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, OneHotEncoder, PowerTransformer, StandardScaler, \
                                  MinMaxScaler, RobustScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold
from sklearn.pipeline import make_pipeline

# For training random forest model
from sklearn.linear_model import ElasticNet, Lasso, LinearRegression
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

# base
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin, clone


from sklearn.metrics import mean_squared_error


In [2]:
pd.set_option("display.max_columns", 100)

In [3]:
# Load the training data
train = pd.read_csv("train.csv", index_col=0)
test = pd.read_csv("test.csv", index_col=0)

# Preview the data
train.head()
train.describe().T

,count,mean,std,min,25%,50%,75%,max
cont0,300000.0,0.527335,0.230599,-0.118039,0.405965,0.497053,0.668060,1.058443
cont1,300000.0,0.460926,0.214003,-0.069309,0.310494,0.427903,0.615113,0.887253
cont2,300000.0,0.490498,0.253346,-0.056104,0.300604,0.502462,0.647512,1.034704
cont3,300000.0,0.496689,0.219199,0.130676,0.329783,0.465026,0.664451,1.039560
cont4,300000.0,0.491654,0.240074,0.255908,0.284188,0.390470,0.696599,1.055424
cont5,300000.0,0.510526,0.228232,0.045915,0.354141,0.488865,0.669625,1.067649
cont6,300000.0,0.467476,0.210331,-0.224689,0.342873,0.429383,0.573383,1.111552
cont7,300000.0,0.537119,0.218140,0.203763,0.355825,0.504661,0.703441,1.032837
cont8,300000.0,0.498456,0.239920,-0.260275,0.332486,0.439151,0.606056,1.040229
cont9,300000.0,0.474872,0.218007,0.117896,0.306874,0.434620,0.614333,0.982922


The next code cell separates the target (which we assign to `y`) from the training features (which we assign to `features`).

In [5]:
# Separate target from features
y = train['target']
features = train.drop(['target'], axis=1)

# Preview features
features.head().T

id,1,2,3,4,6
cat0,B,B,A,B,A
cat1,B,B,A,B,A
cat2,B,A,A,A,A
cat3,C,A,C,C,C
cat4,B,B,B,B,B
cat5,B,D,D,D,D
cat6,A,A,A,A,A
cat7,E,F,D,E,E
cat8,C,A,A,C,A
cat9,N,O,F,K,N


In [6]:
numerical_cols = features.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = features.select_dtypes(include=['object', 'bool']).columns

In [7]:
X_train = features.copy()
X_test = test.copy()

In [2]:
# we will use indexes for the transformers 
numerical_ix = X.columns.get_indexer(numerical_cols)
categorical_ix = X.columns.get_indexer(categorical_cols)

NameError: name 'X' is not defined

### Helper functions

These are two functions to save and load predictions, they can be wrapped within a class for a better modeling or kept as they are since they are independent of the project setting.


In [12]:

## helper fucntions
def store_oof_predictions(model_name,
                          valid_predictions,
                          test_predictions,
                          X_train,
                          X_test,
                          stacking_level,
                          output_folder,
                          n_folds, 
                          random_state):    
    
    """
    Store oof predictions into files.
    It is safe not to use defaults for this helper to make sure 
    we are storing the right experiement. 
    """
    
    if test_predictions is not None:
        test_df = pd.DataFrame({'Id': X_test.index,
                       model_name: test_predictions})
        test_df.to_csv( f'{output_folder}{os.sep}{model_name}_{n_folds}_test.csv',
                       index=False)

    if valid_predictions is not None:
        valid_df = pd.DataFrame({'Id': X_train.index,
                           model_name: valid_predictions})
        valid_df.to_csv(f'{output_folder}{os.sep}{model_name}_{n_folds}_valid.csv', 
                        index=False)
    
    
def read_oof_predictions(pattern, 
                     models_names, 
                     index='id'):    
    """
    Read files according the pattern in their name
    it returns a dataframe where each column is the prediction of a model
    """
    li = [] 
    for f in glob.glob(pattern):
        file_name = f.split(os.sep)[-1] # get file name only
        if file_name.lower().startswith(tuple(s.lower() for s in models_names)): # get only required models
            df = pd.read_csv(f, index_col=0)
            li.append(df)
    
                                   
    return pd.concat(li, axis=1)
    

### Experiment 
Since everything boiled down to stacking (even if we have a single level), the experiment class will handle the organization of the resulted files from the test: test and oofs predictions. Therefore, assuming the project has the following structure with a folder called **experiments** we can save our tests in this folder. This is what this class will do. Another important thing is to create the CV folds, and use them. Currently the class supports one seed only, but that can be changed easily by creating a list of CV indexes instead of a single one. I will add that in the next round.

```
    ML30_project
    │   README.md
    │
    └───notebooks
    │   ...
    │
    └───experiments
    │   │   
    │   │
    │   └───experiment_1
    │   │   │   
    │   │   └───level_1
    │   │   │   xgb_test.csv
    │   │   │   xgb_test.csv
    │   │   │   ...
    │   │   └───level_2
    │   │   │   ...
    │   │   └───level_3
    │   │   │   ...
    │   │   └───level_...
    │   │
    │   │
    │   └───experiment_...
```


>The code that generated the results is important to save too, but that can be done easily by creating a new version of the notebook or copying notebook with the CV_LB results.

>This class is so important when running notebooks in our computers since Kaggle has a nice notebook management system which saves outputs as well.



In [3]:

class Experiment():
    def __init__(self,
                 title='Experiment',
                 n_folds=3,
                 random_state=42,
                 shuffle=True,
                 use_different_random_states=True,
                 main_folder=os.getcwd()):
        
        self.title = title
        self.n_folds = n_folds
        self.random_state = random_state
        self.shuffle = shuffle
        self.use_different_random_states = use_different_random_states
        self.main_folder = main_folder
        
        
    def calc_folds_indexes(self, X, y, sampler=KFold):
        """
        sampler: can be KFold,  StratifiedKFold, or any sampling class  
        """
        
        folds = sampler(n_splits=self.n_folds, 
                        random_state=self.random_state,
                        shuffle=self.shuffle)
        self.folds_idxs = list(self.folds.split(X, y))
        
        
        
    def join_folder(self, folder=None):
            
        # create time stamp and subfolder with the current time stamp
        if folder is not None: # if folder is specified
            self.output_folder = folder 
        else: # create a folder with the time stamp
            time_stamp = datetime.now().isoformat(' ', 'seconds')
            self.output_folder = self.title + ' ' + time_stamp.replace(':', '-')
            Path(f'{self.main_folder}{os.sep}{self.output_folder}').mkdir(parents=True, exist_ok=True)
        
     
    def to_dict(self):
        return dict({"title": self.title,
                    "n_folds": self.n_folds,
                    "random_state": self.random_state,
                    "use_different_random_states": self.use_different_random_states,
                    "main_folder": self.main_folder,
                    "output_folder": self.output_folder})
        
    def __str__(self):
        
        return str(self.to_dict())

NameError: name 'os' is not defined

### ModelWrapper 
This class role is to avoid coding multiple classes for each model (or model types). We can see that models can actually be categorized into different categories, where some models accept more parameters than the others. For instance xgboost can use an evaluation set to determine the stopping round number, while Lasso does not accept such extra parameters.

Thanks to the flexibility of python and the design of the base models, we can wrap the model and `wrapper` to do what the model should do. In fact, we can easily stretch this class to support sklearn pipelines.


In [14]:
class ModelWrapper():
    def __init__(self, 
                 model,
                 name,
                 uses_eval_set=False,
                 fit_params={}):
        
        self.model = model
        self.name = name
        
        self.uses_eval_set = uses_eval_set
        self.fit_params = fit_params # any extra params for the 'fit' function
                
    def fit(self, X, y, eval_set=None):
        if self.uses_eval_set:
            self.model.fit(X, y, eval_set=eval_set, **(self.fit_params))
        else:
            self.model.fit(X, y, **(self.fit_params)) 
        return self
    

    def predict(self, X):
        return self.model.predict(X)
        
    def clone_me(self, random_state=None):
        wrapper = ModelWrapper(model=clone(self.model), # clone from sklean.base
                               name=self.name, 
                               uses_eval_set=self.uses_eval_set,
                               fit_params=self.fit_params)
        wrapper.name = self.name
        if random_state is not None:
            wrapper.set_random_state(random_state)
        
        return wrapper
    
    def set_random_state(self, random_state):
        if hasattr(self.model, 'random_state'):
            self.model.random_state = random_state
        elif hasattr(self.model, 'random_seed'):
            self.model.random_seed = random_state
            
    def get_random_state(self):
        if hasattr(self.model, 'random_state'):
            return self.model.random_state 
        elif hasattr(self.model, 'random_seed'):
            return self.model.random_seed
    
        

### ModelTrainer
This class role is to the a model and calculate the oofs and the test results.


In [4]:
class ModelTrainer():
    def __init__(self,
                  model: ModelWrapper):
        
        self.model = model
        
    def calc_oofs(self,
                  X, y,
                  X_test,
                  folds,
                  transformer=None,
                  verbose=False,
                  use_different_random_states=True):
        """
        Return the oofs predictions and the meta features (test predictions)
        """
        
        test_predictions = 0
        oof_predictions = np.zeros_like(np.array(y))
        valid_mean_score = [] 
                
        for fold, (train_ix, valid_ix) in enumerate(folds.split(X, y)):
            X_train, X_valid = X[train_ix], X[valid_ix]
            y_train, y_valid = y[train_ix], y[valid_ix]
                        
            
            # transform input
            if transformer is not None:
                X_train = transformer.fit_transform(X_train)
                X_valid = transformer.transform(X_valid)

            
            # check if we train each fold on differently initialized clone
            if use_different_random_states:
                model = self.model.clone_me(random_state=fold)
            else:
                model = self.model.clone_me()
            
            # fit the model
            if model.uses_eval_set:
                model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)])
            else:
                model.fit(X_train, y_train)
                
            ## predictions
            # on the validation set
            valid_predications = model.predict(X_valid)
            score = mean_squared_error(valid_predications, y_valid, squared=False)
            valid_mean_score.append(score)
            oof_predictions[valid_ix] = valid_predications
            
            # on the test set
            # transform it first on a copy
            if transformer is not None:
                X_test_ = transformer.transform(X_test)
            else:
                X_test_ = X_test
            test_predictions += model.predict(X_test_) / folds.n_splits
            
            if verbose:
                print('Fold:{} score:{:.4f}'.format(fold + 1, score))
        
        if verbose:
            print('Average score:{:.4f} ({:.4f})'.format(np.mean(valid_mean_score), np.std(valid_mean_score) ))
    
        return oof_predictions, test_predictions

NameError: name 'ModelWrapper' is not defined

### LevelTrainer
This class role glues, everything in one place and train each model in a given level.


In [17]:
class LevelTrainer():
    
    def __init__(self,
                 experiment, # Experiment,
                 models,      # ModelWrapper
                 transformer, # data transformer
                 level_num=1):
        
        self.level_num = level_num
        self.experiment = experiment
        self.models = models
        self.transformer = transformer
        
    def train(self, X_df, y_df, X_test_df, verbose=True):
        
        assert models is not None
        

        for model_wrapper in self.models:
            
            if verbose:
                print('-'*30)
                print(f'Model:{model_wrapper.name}')
                print('-'*30)
                
            trainer = ModelTrainer(model_wrapper)
            oof_preds, test_preds = trainer.calc_oofs(X_df.values, y_df.values,
                                                      X_test_df.values,
                                                      transformer=self.transformer,
                                                      folds=self.experiment.folds,
                                                      verbose=verbose,
                                                      use_different_random_states=self.experiment.use_different_random_states)
            # store predictions
            store_oof_predictions(model_name=model_wrapper.name,
                                  valid_predictions=oof_preds,
                                  test_predictions=test_preds,
                                  X_train=X_df,
                                  X_test=X_test_df,
                                  stacking_level=self.level_num, 
                                  n_folds=self.experiment.n_folds, 
                                  random_state=model_wrapper.get_random_state(),
                                  output_folder=f'{self.experiment.main_folder}{os.sep}{self.experiment.output_folder}')
            if verbose:
                print('-'*30)

### Hyperparameters

Here goes the paramaters of each model. These can actually be stored in an external JSON file.


In [7]:
# Lasso
lasso_params = {
                alpha: 0.00005
}


# Elastic Net
enet_params = {
               alpha: 0.00005, 
               l1_ratio: .9
}

# extra-tree
et_params = {
    'n_jobs': -1,
    'n_estimators': 100,
    'max_features': 0.5,
    'max_depth': 12,
    'min_samples_leaf': 2,
}

# random forest

rf_params_2 = {
    'n_jobs': -1,
    'n_estimators': 500,
    'max_depth': 5
}


rf_params_1 = {
    'n_jobs': -1,
    'n_estimators': 100,
    'max_features': 0.2,
    'max_depth': 8,
    'min_samples_leaf': 2
}

# gradient boosting 
gb_params = {
    'n_estimators': 500,
     'max_depth': 3
}

# xgboost

# Using optuna but still rough
xgb_params_2 = {
#                  'tree_method': 'gpu_hist', 
#                  'gpu_id': 0, 
#                  'predictor': 'gpu_predictor',
    
                 'max_depth': 5,
                 'learning_rate': 0.021252439960137114,
                 'n_estimators': 13500,
                 'subsample': 0.62,
                 'booster': 'gbtree',
                 'colsample_bytree': 0.1,
                 'reg_lambda': 0.1584605320779582,
                 'reg_alpha': 15.715145781076245,
                 'n_jobs': -1
}

xgb_params_1 = {
            'random_state': 1, 
            # gpu
    #         'tree_method': 'gpu_hist', 
    #         'gpu_id': 0, 
    #         'predictor': 'gpu_predictor',
            # cpu
            'n_jobs': -1,
            'booster': 'gbtree',
            'n_estimators': 10000,
            # optimized params
            'learning_rate': 0.03628302216953097,
            'reg_lambda': 0.0008746338866473539,
            'reg_alpha': 23.13181079976304,
            'subsample': 0.7875490025178415,
            'colsample_bytree': 0.11807135201147481,
            'max_depth': 3,
            #'min_child_weight': 6
}


# catboost
catb_params_2 = {'iterations': 20000,
              'learning_rate': 0.45,
              'loss_function': "RMSE",
              'random_state': 0,
              'verbose': 0,
              'thread_count': -1,
              'depth': 1}


catb_params_1 = {'iterations': 6800,
              'learning_rate': 0.93,
              'loss_function': "RMSE",
              'random_state': 42,
              'verbose': 0,
              'thread_count': -1,
              'depth': 1,
              'l2_leaf_reg': 3.28}


# using optuna
params_lgb = {
             "n_estimators": 10000,
             'metric':'rmse',
             "objective": "regression",
             'max_depth': 12, 
             'subsample': 0.587082286344555, 
             'colsample_bytree': 0.2157299997089329, 
             'learning_rate': 0.01270518267668901,
             'reg_lambda': 36.78473508062132,
             'reg_alpha': 14.155146595119032, 
             'min_child_samples': 6, 
             'num_leaves': 34, 
             'max_bin': 914,
             'cat_smooth': 26,
             'n_jobs': -1,
             'cat_l2': 0.020257336654989123
        }

NameError: name 'alpha' is not defined

### These are model/task dependent parameters

In [5]:
# external hyperparamaters

### fit function hyperparamaters
# some models require special paramaters like early stoping in xgboost and lgbm
fit_params = {'early_stopping_rounds': 300,
                  'verbose': False}

### application/implementation paramaters
# These paramaters are implementation dependent 
app_params = {'uses_eval_set':True}



### Models
Here we define goes all models.

In [20]:
lrg = LinearRegression()
# lasso
lasso = Lasso(**lasso_params)

# Elastic net
e_net = ElasticNet(**enet_params)

# KNeighborsRegressor
knn =  KNeighborsRegressor()

# extra-tree
extree = ExtraTreesRegressor(**et_params)

# random forest
rfr = RandomForestRegressor(**rf_params_2)

# gradient boosting
gb = GradientBoostingRegressor(**gb_params)

#lgbm
lgb = LGBMRegressor(**params_lgb_1)

# variants

# xgboost
xgb_1 =  XGBRegressor(**xgb_params_1)
xgb_2 =  XGBRegressor(**xgb_params_2)

#catboost
catb_1 = CatBoostRegressor(**catb_params_1)

In [21]:
# compile all settings in one dictionary, 
# we can store/load it then to a JSON file
models = {'LinearRegression': {"model":lrg, "fit_kwargs":None, "app_params": None},
          'Lasso': {"model":lasso, "fit_kwargs":None, "app_params": None},
          'ElasticNet': {"model": e_net, "fit_kwargs":None, "app_params": None},
          'ExtraTreesRegressor': {"model": extree, "fit_kwargs":None, "app_params": None},
          'RandomForestRegressor': {"model": rfr, "fit_kwargs":None, "app_params": None},
          'GradientBoostingRegressor': {"model": gb, "fit_kwargs":None, "app_params": None},
          'XGBRegressor-1': {"model": xgb_1, "fit_kwargs":fit_params_300, "app_params": app_params},
          'XGBRegressor-2': {"model": xgb_2, "fit_kwargs":fit_params_300, "app_params": app_params},
          'CatBoostRegressor-1': {"model": catb_1, "fit_kwargs": fit_params_300, "app_params": fit_params_300},
          'LGBMRegressor': {"model": lgb, "fit_kwargs":fit_params_300, "app_params": app_params}
          # we can add any number of models here 
        }

In [22]:
models.keys()

dict_keys(['LinearRegression', 'Lasso', 'ElasticNet', 'ExtraTreesRegressor', 'RandomForestRegressor', 'GradientBoostingRegressor', 'XGBRegressor-1', 'XGBRegressor-2', 'XGBRegressor-3', 'XGBRegressor-4', 'XGBRegressor-5', 'CatBoostRegressor-1', 'CatBoostRegressor-2', 'CatBoostRegressor-3', 'LGBMRegressor'])

### Stacking

Here goes the actual stacking procedure. 
   - We first define the architecture, and setup the a session.
   - Define the stack. That is, the models and transformers in the levels

In [53]:
# settings: experiment and stacking architecutre
session_folder = "Experiments/new_session"
X_train_, X_test_ = X_train, X_test # initialized to the input


# any special transformers for any level
level_1_transformers = [('cat', OrdinalEncoder(), categorical_ix), ('num', MinMaxScaler(), numerical_ix)]
level_1_col_transform = ColumnTransformer(transformers=level_1_transformers)


level_2_transform = None # MinMaxScaler() PowerTransformer() 

stack = [ {"level-id": "level-1", 
           "models": [
                     'CatBoostRegressor-1',
                     'XGBRegressor-1',
                     # we can add any model here
                    ],
            "use-only": None, # to select subset of models in the next level
            "n_folds": 5,
            "folder": "level_1", 
            "transformer": level_1_col_transform, 
            "frozen": False
            },
               
           {"level-id": "level-2",
            "models": [
                       'RandomForestRegressor'
                      ],
            "use-only": None, 
            "n_folds": 5,
            "folder": "level_2",
            "transformer": level_2_transform,
            "frozen": False
          },

# we can add any number of levels here
#           {"level-id": "level-3",
#             "models": ['Lasso',
#                        'XGBRegressor-1', 
#                        'CatBoostRegressor-1',
#                        'LGBMRegressor',
#                        'RandomForestRegressor'],
#             "use-only": None, 
#             "n_folds": 5,
#             "folder": "level_3",
#             "transformer": level_2_transform 
#           }
        
        ]
         

- Loop through each level in the stack

In [54]:
# if we need to freeze a level
# I think we can also add a feature easily to freeze a model at a level

# run the stack
for  level in stack:
    
    print('-'*50)
    print(f'Current Level: {level["level-id"]}')
    print('-'*50)
    
    # get current path
    output_path = os.getcwd()
    
    # get models 
    # if level_1_models is set to 'all' use all models
    if level['models'][0].lower() == 'all':
        level_models_names = models.keys()
    else: 
        level_models_names = level['models']

    # create experiment
    ml30_experiment = Experiment(title='ML 30 days',
                        n_folds=level['n_folds'],
                        random_state=42,
                        shuffle=True,
                        use_different_random_states=True,
                        main_folder=f'{output_path}{os.sep}{session_folder}')

    ml30_experiment.create(use_folder=level['folder'], resume=True)

    # create models
    model_wrappers = []
    for model_name in level_models_names:

        # get paramaters  
        model = models[model_name]
        fit_kwargs = models[model_name]['fit_kwargs']
        app_params = models[model_name]['app_params']

        model_wrapper = ModelWrapper(model=model['model'], name=model_name)
        if fit_kwargs is not None:
            model_wrapper.fit_params = fit_kwargs
        if app_params is not None:
            model_wrapper.uses_eval_set = app_params['uses_eval_set']
        model_wrappers.append(model_wrapper)

        
    # train the level
    if freeze is not None:
        if level['level-id'] not in freeze:  # escape any trained level   
            level_trainer = LevelTrainer(experiment=ml30_experiment, 
                                 models=model_wrappers, 
                                 transformer=level['transformer'])

            level_trainer.train(X_df=X_train_, y_df=y, X_test_df=X_test_)
        else:
            print('This level is already trained')
            

    # collect results    
    fold_id = ml30_experiment.n_folds
    folder =f'{session_folder}{os.sep}{ml30_experiment.output_folder}'
                            
    print(f"{folder}{os.sep}*{fold_id}_valid.csv")
    
    # new features 
    X_train_ = read_oof_predictions(pattern=f"{folder}{os.sep}*{fold_id}_valid.csv",
                               models_names=level_models_names)
    X_test_ = read_oof_predictions(pattern=f"{folder}{os.sep}*{fold_id}_test.csv",
                              models_names=level_models_names)
    
    display(X_train_.head(10))
    display(X_test_.head(10))


--------------------------------------------------
Current Level: level-1
--------------------------------------------------
This level is already trained
Experiments/new_session/level_1/*5_valid.csv


,XGBRegressor-1,CatBoostRegressor-1
Id,,
1,8.484386,8.477987
2,8.435083,8.315481
3,8.215256,8.254283
4,8.350307,8.387012
6,8.248346,8.270340
7,8.078706,8.038880
8,8.282299,8.271361
9,8.346571,8.356183
10,8.389336,8.405265


,XGBRegressor-1,CatBoostRegressor-1
Id,,
0,8.079207,8.089023
5,8.393541,8.409556
15,8.415014,8.434104
16,8.500801,8.562139
17,8.134493,8.154888
19,8.306716,8.282289
20,8.542321,8.476730
21,7.881266,7.846250
23,8.073944,8.140942


In [55]:
# final results
X_test_.head(10)

,XGBRegressor-1,CatBoostRegressor-1
Id,,
0,8.079207,8.089023
5,8.393541,8.409556
15,8.415014,8.434104
16,8.500801,8.562139
17,8.134493,8.154888
19,8.306716,8.282289
20,8.542321,8.476730
21,7.881266,7.846250
23,8.073944,8.140942


### Submit the results

In [65]:
predictions = X_test_.values
# Save the predictions to a CSV file
output = pd.DataFrame({'Id': X_test.index,
                       'target': predictions})
output.to_csv('submission.csv', index=False)

In [66]:
output.head(20)

,Id,target
0,0,8.072493
1,5,8.420695
2,15,8.421442
3,16,8.501886
4,17,8.157393
5,19,8.329672
6,20,8.523518
7,21,7.858731
8,23,8.060452
9,29,8.331367
